In [0]:
%load_ext autoreload
%autoreload 2

In [2]:
# for Google Colab only: Mount my Google Drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [3]:
pwd

'/gdrive'

In [4]:
# for Google Colab only: Navigate to my snaug project folder
cd 'My Drive/Colab Notebooks/snaug'

/gdrive/My Drive/Colab Notebooks/snaug


In [5]:
# peek at current system Path settings
import sys
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [6]:
# let's add my project path so that I can load my custom written
# library functions saved in '/lib' 
sys.path.append("/gdrive/My Drive/Colab Notebooks/snaug")
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/gdrive/My Drive/Colab Notebooks/snaug']

In [7]:
#
# Load trained models to generate new text
#

import string
import textwrap
import pickle
import sys
from random import randint

from keras.models import load_model

from lib.nlplstm_class import (TFModelLSTMCharToken, TFModelLSTMWordToken, 
                               TFModelLSTMWord2vec) 
from lib.data_common import (load_doc, save_doc, clean_doc, prepare_char_tokens,
                             build_token_lines, prepare_text_tokens, load_word2vec,
                             sample_predict, generate_seq_of_chars, generate_seq_of_words)
from lib.data_common import pathfinder_textfile, fixed_length_token_textfile

Using TensorFlow backend.


In [0]:
# parameters for character tokenisation model
maxlen = 40
step = 3

In [0]:
# load fixed-length lines of tokens
doc = load_doc(fixed_length_token_textfile)
lines = doc.split('\n')

In [0]:
# generate word_seed_text for word tokenisation models
# generate separate char_seed_text for char tokenisation model, using
# maxlen rightmost characters of word seed text
word_seed_text = lines[randint(0,len(lines))]
char_seed_text = word_seed_text[-maxlen:]

In [0]:
# LSTM model that uses character tokenisation  
#
# load document
text = load_doc(pathfinder_textfile).lower()

In [12]:
# tokenize character data and separate into features and target for LSTM training
X, y, char2indices, indices2char, num_unique_char = prepare_char_tokens(text, maxlen, step)

corpus length: 76633
total chars: 57
number of sequences: 25531
number of next_chars: 25531


In [14]:
# load trained model
textgen_model_1 = load_model('./model/pathfinder_chartoken_model_50_epoch_withcuda_model.h5')





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [15]:
# select seed text
#start_index = randint(0, len(text) - maxlen - 1)
#seed_text = text[start_index: start_index + maxlen]
print()
print('> using character tokenisation')
print('> seed text:')
print(textwrap.fill('%s' % (char_seed_text), 80) + '\n')


> using character tokenisation
> seed text:
rs coming through absalom the pathfinder



In [0]:
# generate new text
#temperature_table = [0.2, 0.7,1.2] 
#temperature_table = [0.07, 0.1, 0.5, 0.7] 
temperature_table = [0, 0.7] 

In [17]:
for temperature in temperature_table:
    generated = generate_seq_of_chars(textgen_model_1, num_unique_char, char2indices, 
                    indices2char, char_seed_text, maxlen, 300, temperature)    
    print(">> generated text (temperature: {})".format(temperature))
    print(textwrap.fill('%s' % (generated), 80))
    print()

>> generated text (temperature: 0)
 society and gave her per and stracks, and a gearby secked dedeaty ar insthe
rosk again for sheeter, and a scong them behind the leash the pcs may of may
noter her ad and present on streem, and creatures a stealth. mayor secter the
riveled of forts. in or meas the pcs may or montrovale’s res” in the

>> generated text (temperature: 0.7)
 society and gave her adpead bagl mors age reasy and persention the mimic, the
pcs from her the safe mished of the pcs may of may bazs durains the cimbat.  she
tap an and dwarves for grambes with the ghelle’s can be pider the rains, and
combat yom sout of hen sumpeed with a successful dc 20 deracted



In [0]:
# 
# Loading text data that uses word tokenisation
#

In [0]:
# tokenize and separate into features and target
X, y, seq_length, vocab_size, tokenizer = prepare_text_tokens(lines)

In [0]:
#
# Word tokenization with word embedding model
#

In [21]:
# load trained model
textgen_model_2 = load_model('./model/pathfinder_wordtoken_model_200_epoch_withcuda_model.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [22]:
# select a seed text
#word_seed_text = lines[randint(0,len(lines))]
print('> using word tokenisation')
print('> seed text:')
print(textwrap.fill('%s' % (word_seed_text), 80) + '\n')

> using word tokenisation
> seed text:
off with the true crosier eberius rages for a few painful moments before
muttering to himself that need to find another team to hunt down the bloody
knuckles before the relic disappears from sandpoint for good having purchased
information from a roving band of varisian wanderers coming through absalom the
pathfinder



In [0]:
# generate new text
temperature_table = [0, 0.7]

In [24]:
for temperature in temperature_table:
    generated = generate_seq_of_words(textgen_model_2, tokenizer, seq_length, 
                    word_seed_text, 100, temperature)
    print(">> generated text (temperature: {})".format(temperature))
    print(textwrap.fill('%s' % (generated), 80))
    print()

>> generated text (temperature: 0)
society has tasked the pcs with investigating a set of ruins in northern varisia
believing the site to be a lost thassilonian outpost the society wants the pcs
to bring back any artifacts relating to the ancient culture of the runelord the
image grants them a rare and valuable thassilonian relic as the pcs continue to
explore the ruins the self imposed goblin guardians of the rockwhelp tribe
assault them the pcs make their way into the depths of the lurkwood in northern
varisia and come upon a series of ruins there they trigger a long dormant
programmed illusion which



/gdrive/My Drive/Colab Notebooks/snaug/lib/data_common.py:143: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


>> generated text (temperature: 0.7)
society has tasked the pcs with investigating a set of ruins in northern varisia
believing the site to be a lost thassilonian outpost the society wants the pcs
to bring back any artifacts relating to the ancient culture of the runelord the
image grants them a rare and valuable thassilonian relic as the pcs continue to
explore the ruins the self imposed goblin guardians of the rockwhelp tribe
assault them the pcs make their way into the depths of the lurkwood in northern
varisia and come upon a series of ruins there they trigger a long dormant
programmed illusion which



In [0]:
#
# Word2vec pre-trained model
#

In [27]:
# load trained model
textgen_model_3 = load_model('./model/pathfinder_wordtoken_w2v_model_50_epoch_withcuda_model.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:310: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [28]:
# select a seed text
#word_seed_text = lines[randint(0,len(lines))]
print('> using word tokenisation and pre-trained model')
print('> seed text:')
print(textwrap.fill('%s' % (word_seed_text), 80) + '\n')

> using word tokenisation and pre-trained model
> seed text:
off with the true crosier eberius rages for a few painful moments before
muttering to himself that need to find another team to hunt down the bloody
knuckles before the relic disappears from sandpoint for good having purchased
information from a roving band of varisian wanderers coming through absalom the
pathfinder



In [0]:
# generate new text
temperature_table = [0, 1.1]

In [30]:
for temperature in temperature_table:
    generated = generate_seq_of_words(textgen_model_3, tokenizer, seq_length, 
                    word_seed_text, 100, temperature)
    print(">> generated text (temperature: {})".format(temperature))
    print(textwrap.fill('%s' % (generated), 80))
    print()

>> generated text (temperature: 0)
society has tasked the pcs with investigating a set of ruins in northern varisia
believing the site to be a lost thassilonian outpost the society wants the pcs
to bring back any artifacts relating to her lair but he arrived the venture
captain needed to know she has the codex and for a fairly modest price she will
let the society have it but she needs to be discrete about it heidmarch
suspicious that montrovale might be in trouble decided to act quickly even if
her instincts told her that getting involved with academae politics could be
risky knowing that



/gdrive/My Drive/Colab Notebooks/snaug/lib/data_common.py:143: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds) / temperature


>> generated text (temperature: 1.1)
society has tasked the pcs with investigating a set of ruins in northern varisia
believing the site to be a lost thassilonian outpost the society wants the pcs
to bring back any artifacts relating to her lair for the dwarves went east to
area if any pcs examine the boot prints have each searching pc make a perception
check any pc who gets a or higher finds a dozen hand sized dead burned spiders
in the blackened dirt any pc who gets a or higher spots the broken glass
remnants of a flask of fire the walls of the mine

